In [3]:
import os
import dotenv
dotenv.load_dotenv(".env.local")

from supabase import create_client, Client
import time
from copy import deepcopy

import re
import json
from urllib.request import urlopen
import requests
import pandas as pd
import plotly.express as px
px.set_mapbox_access_token(os.environ["MAPBOX_TOKEN"])
from datetime import datetime, timedelta

In [4]:
client = create_client("https://wvklzyidkkrskdgquocj.supabase.co", os.environ["SUPABASE_KEY"])
response = client.table('visits').select("*").execute()

2024-03-03 15:41:07,770:INFO - HTTP Request: GET https://wvklzyidkkrskdgquocj.supabase.co/rest/v1/visits?select=%2A "HTTP/1.1 200 OK"


In [5]:
visits = deepcopy(response.data)
visits_df = pd.DataFrame(visits)
visits_df['created_at'] = pd.to_datetime(visits_df['created_at'])
visits_df

,id,created_at,ip_address,route,user_agent,is_bot,referrer
0,7,2024-02-27 04:41:20.067741+00:00,64.23.187.48,/,None,None,None
1,8,2024-02-27 04:41:20.795991+00:00,64.23.187.48,/,None,None,None
2,9,2024-02-27 04:41:21.213661+00:00,172.69.23.187,/,None,None,None
3,10,2024-02-27 04:42:53.205514+00:00,76.99.220.36,/,None,None,None
4,11,2024-02-27 04:43:51.899723+00:00,172.69.22.42,/,None,None,None
...,...,...,...,...,...,...,...
422,427,2024-03-03 13:05:38.378563+00:00,83.229.85.81,/,python-requests/2.31.0,true,None
423,428,2024-03-03 13:51:14.315301+00:00,17.241.219.73,/cooking,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5...,true,None
424,429,2024-03-03 17:44:54.681990+00:00,182.42.104.32,/,Mozilla/5.0 (iPhone; CPU iPhone OS 13_2_3 like...,false,https://www.isaacwasserman.com/
425,430,2024-03-03 18:32:28.828089+00:00,116.205.133.55,/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,false,None


In [6]:
maximum_session_gap = 30
sessions = []
for ip_address in visits_df['ip_address'].unique():
    visitors_visits = visits_df[visits_df['ip_address'] == ip_address]
    visitors_visits = visitors_visits.sort_values('created_at')
    session = []
    for i in range(1, len(visitors_visits)):
        if session == []:
            session.append(visitors_visits.iloc[i])
        previous_visit = session[-1]
        current_visit = visitors_visits.iloc[i]
        time_difference = (current_visit['created_at'] - previous_visit['created_at']).seconds / 60
        if time_difference > maximum_session_gap:
            sessions.append(session)
            session = []
        session.append(current_visit)
    if session != []:
        sessions.append(session)

sessions_df = {
    "ip_address": [],
    "start_time": [],
    "end_time": [],
    "visit_ids": []
}

for session in sessions:
    sessions_df["ip_address"].append(session[0]['ip_address'])
    sessions_df["start_time"].append(session[0]['created_at'])
    sessions_df["end_time"].append(session[-1]['created_at'])
    sessions_df["visit_ids"].append([visit['id'] for visit in session])

sessions_df = pd.DataFrame(sessions_df)

In [19]:
response = client.table('visitors').select("*").execute()
visitors = deepcopy(response.data)

2024-03-03 15:47:30,911:INFO - HTTP Request: GET https://wvklzyidkkrskdgquocj.supabase.co/rest/v1/visitors?select=%2A "HTTP/1.1 200 OK"


In [20]:
visitors_df = pd.DataFrame(visitors)
new_visitors = []
ip_addresses_in_sessions = sessions_df['ip_address'].unique()
for ip_address in ip_addresses_in_sessions:
    if ip_address in visitors_df['ip_address'].values:
        continue
    ip_info = json.load(urlopen(f"https://ipinfo.io/{ip_address}/json"))
    loc = ip_info.get('loc', None)
    if loc:
        lat, lon = loc.split(',')
        ip_info['lat'] = float(lat)
        ip_info['lon'] = float(lon)
    else:
        ip_info['lat'] = None
        ip_info['lon'] = None
    visitor = {
        "ip_address": ip_address,
        "city": ip_info.get('city', None),
        "region": ip_info.get('region', None),
        "country": ip_info.get('country', None),
        "postal": ip_info.get('postal', None),
        "lat": ip_info.get('lat', None),
        "lon": ip_info.get('lon', None),
        "timezone": ip_info.get('timezone', None),
        "hostname": ip_info.get('hostname', None),
        "org": ip_info.get('org', None),
        "is_bot": visits_df[visits_df['ip_address'] == ip_address].iloc[-1]['is_bot']
    }
    new_visitors.append(visitor)

response = client.table('visitors').insert(new_visitors).execute()

2024-03-03 15:47:32,314:INFO - HTTP Request: POST https://wvklzyidkkrskdgquocj.supabase.co/rest/v1/visitors "HTTP/1.1 201 Created"


In [21]:
len(visitors_df)

55

In [25]:
dots_df = sessions_df.copy()
dots_df['lat'] = None
dots_df['lon'] = None
dots_df['city'] = None
dots_df['is_bot'] = None
dots_df['referrer'] = None
dots_df['hostname'] = None
for i, row in dots_df.iterrows():
    lat = visitors_df[visitors_df['ip_address'] == row['ip_address']].iloc[0]['lat']
    lon = visitors_df[visitors_df['ip_address'] == row['ip_address']].iloc[0]['lon']
    city = visitors_df[visitors_df['ip_address'] == row['ip_address']].iloc[0]['city']
    is_bot = visitors_df[visitors_df['ip_address'] == row['ip_address']].iloc[0]['is_bot']
    referrer = visits_df[visits_df['id'].isin(row['visit_ids'])].iloc[0]['referrer']
    hostname = visitors_df[visitors_df['ip_address'] == row['ip_address']].iloc[0]['hostname']
    dots_df.at[i, 'lat'] = lat
    dots_df.at[i, 'lon'] = lon
    dots_df.at[i, 'city'] = city
    dots_df.at[i, 'is_bot'] = is_bot
    if not referrer:
        referrer = ""
    dots_df.at[i, 'referrer'] = referrer
    if not hostname:
        hostname = ""
    dots_df.at[i, 'hostname'] = hostname

num_last_days = 4

dots_df['start_time'] = pd.to_datetime(dots_df['start_time'])
timezone = dots_df['start_time'].dt.tz
seven_days_ago = datetime.now(timezone) - timedelta(days=num_last_days)
dots_df = dots_df[dots_df['start_time'] > seven_days_ago]

fig = px.scatter_mapbox(dots_df, lat="lat", lon="lon", color="is_bot", hover_name="ip_address", hover_data=["city", "referrer", "hostname", "start_time", "end_time",], zoom=1)
fig.update_layout(mapbox_style="open-street-map")
fig.show()